# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [77]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
cf.go_offline()

In [78]:
data = pd.read_excel('../Data/Online Retail.xlsx')

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [79]:
data_print = data[data.Country != "United Kingdom"]
data_print1 = data_print.loc[(data_print["InvoiceDate"]>= '2011-4-1') & (data_print["InvoiceDate"]<= '2011-4-30')]
data_print1.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
724,549667,2011-04-11 12:20:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
3320,550899,2011-04-21 12:07:00,22752,SET 7 BABUSHKA NESTING BOXES,2,8.50,17.00,13505,Switzerland
4880,550527,2011-04-19 10:48:00,84879,ASSORTED COLOUR BIRD ORNAMENT,40,1.69,67.60,12476,Germany
4882,550620,2011-04-19 13:39:00,84879,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,12585,Germany


In [80]:
d=data_print1.groupby('Country').sum()
d=d[['Quantity','Revenue']]
fig = d.iplot(kind='bar')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [94]:
data_print2 = data[data.Country == "France"]
data_print2 = data_print2.loc[(data_print["InvoiceDate"]>= '2011-1-1') & (data_print["InvoiceDate"]<= '2011-5-31')]
dataprint2 = data_print2[['Quantity','Revenue',"InvoiceDate"]]
dataprint2 = dataprint2.resample("D", on="InvoiceDate").sum().sort_values(by="InvoiceDate")

In [95]:
dataprint2.iplot(kind="line")

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [56]:
data_print3 = data[data.Description == "PARTY BUNTING"]
d3=data_print3.groupby('Country').agg({'Quantity':'mean','UnitPrice':'mean'})
d3=d3.reset_index()
d3.iplot( x='Quantity', y='UnitPrice',categories='Country',
           xTitle='Quantity', yTitle='UnitPrice',
           title='PARTY BUNTING by country')

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [120]:
#data_print4 = data[data.Country.str.contains("Germany|Netherlands|EIRE|France")] Para conocimiento propio
paises = ["France","EIRE","Germany","Netherlands"]

for p in paises:
    df4 = data[data.Country == p]
    df4 = df4["Quantity"]
    df4.iplot(kind='hist')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [142]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']
country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [143]:
data_print5 = data[data.Country.str.contains("Germany|Netherlands|EIRE|France")] 
data_print5 = data_print5[data_print5.Description.str.contains("JUMBO BAG RED RETROSPOT|CREAM HANGING HEART T-LIGHT HOLDER|REGENCY CAKESTAND 3 TIER")]

In [145]:
data_print5 = data_print5.pivot_table(values='Revenue', columns='Description', 
                              index='Country', aggfunc='sum') #Saqué el pivot table!

In [146]:
data_print5.iplot(kind='bar')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [180]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [230]:
@interact(Selection=['Day', 'Month', 'Year',])

def linechart(Selection):
    data8 = uk.pivot_table(values='Quantity',
                            index=Selection, aggfunc='sum').reset_index()
    data8.iplot(kind='line', x=Selection)


interactive(children=(Dropdown(description='Selection', options=('Day', 'Month', 'Year'), value='Day'), Output…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [226]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

In [227]:
products = products.reset_index()

In [233]:
products = products[0:300]

In [236]:
products.iplot( x='InvoiceNo', y='CustomerID',categories='Description',
           xTitle='InvoiceNo', yTitle='CustomerID',
           title='Título de esta movida')

In [241]:
@interact(bins = (8, len(products['InvoiceNo'].unique())))
def hist(bins):
    products['InvoiceNo'].iplot(kind='scatter', bins=bins, title='Tenure Distribution') # esto no es así

interactive(children=(IntSlider(value=77, description='bins', max=147, min=8), Output()), _dom_classes=('widge…

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.